In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# Set up matplotlib defaults: larger images, gray color map
import matplotlib
matplotlib.rcParams.update({
    'figure.figsize': (10, 10),
    'image.cmap': 'gray'
})

In [ ]:
from skimage import io
image = io.imread('../data/diatom-wild-032.jpg')

plt.imshow(image);

In [ ]:
pores = image[:690, :]

plt.imshow(pores);

In [ ]:
from skimage import filters
from skimage import util

denoised = filters.median(util.img_as_float(pores), behavior='ndimage')

In [ ]:
pores_sqrt = np.sqrt(denoised)
plt.imshow(pores_sqrt);

In [ ]:
pores_inv = 1 - pores_sqrt
plt.imshow(pores_inv);

In [ ]:
from skimage import filters

In [ ]:
thresholded = (pores_sqrt < np.percentile(pores_sqrt, 15))
plt.imshow(thresholded);

In [ ]:
from scipy import ndimage as ndi
from skimage import segmentation, morphology, color

In [ ]:
distance = ndi.distance_transform_edt(thresholded)
local_maxima = morphology.local_maxima(distance)

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(20, 20))

maxi_coords = np.nonzero(local_maxima)

ax.imshow(pores);
plt.scatter(maxi_coords[1], maxi_coords[0]);

In [ ]:
def shuffle_labels(labels):
    """Shuffle the labels so that they are no longer in order.
    This helps with visualization.
    """
    indices = np.unique(labels)
    indices = np.append(
        [0],
        np.random.permutation(indices)
    )
    return indices[labels]

In [ ]:
markers = ndi.label(local_maxima)[0]
labels = segmentation.watershed(-distance, markers)

In [ ]:
f, (ax0, ax1, ax2) = plt.subplots(1, 3, figsize=(20, 5))
ax0.imshow(thresholded)
ax1.imshow(np.log(1 + distance))
ax2.imshow(shuffle_labels(labels), cmap='magma');

In [ ]:
labels_masked = segmentation.watershed(-distance, markers, mask=thresholded, connectivity=2)

In [ ]:
f, (ax0, ax1, ax2) = plt.subplots(1, 3, figsize=(20, 5))
ax0.imshow(thresholded)
ax1.imshow(np.log(1 + distance))
ax2.imshow(shuffle_labels(labels_masked), cmap='magma');

In [ ]:
from skimage import measure

contours = measure.find_contours(shuffle_labels(labels_masked), level=0.5)
plt.imshow(pores)
for c in contours:
    plt.plot(c[:, 1], c[:, 0])